Задание 1

In [3]:
import pandas as pd

In [24]:
movies_data = pd.read_csv('ml-latest-small/movies.csv')
movies_ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [25]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
movies_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [31]:
%%time

def movies_classification(movie_row):
    if movie_row['rating'] <= 2.0:
        return 'low'
    elif movie_row['rating'] <= 4.0:
        return 'mid'
    else:
        return 'high'


movies_ratings['rating_class'] = movies_ratings.apply(movies_classification, axis=1)


CPU times: total: 516 ms
Wall time: 718 ms


In [32]:
movies_ratings.head()

,userId,movieId,rating,timestamp,rating_class
0,1,31,2.5,1260759144,mid
1,1,1029,3.0,1260759179,mid
2,1,1061,3.0,1260759182,mid
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,mid


In [42]:
result_frame = pd.merge(movies_data, movies_ratings).groupby(['movieId', 'title']).agg({
    'rating': 'mean',
    'rating_class': [set]}).reset_index()
result_frame.head()

,movieId,title,rating,rating_class
,,,mean,set
0,1,Toy Story (1995),3.872470,"{high, low, mid}"
1,2,Jumanji (1995),3.401869,"{low, high, mid}"
2,3,Grumpier Old Men (1995),3.161017,"{high, low, mid}"
3,4,Waiting to Exhale (1995),2.384615,"{low, mid}"
4,5,Father of the Bride Part II (1995),3.267857,"{low, mid, high}"
